In [4]:
run ~/Projects/replab/replab_addpaths(2,1); % replace by your own path
install_sdpt3;

VPI is already in the path
MOxUnit is already in the path
YALMIP is already in the path
SDPT3 is already in the path
MOcov is already in the path

---------------------------------------------------------------------------
SDPT3 installation script
   Directory: /home/denis/Projects/replab/external/SDPT3
   Matlab 9.4.0.813654 (R2018a) on GLNXA64
---------------------------------------------------------------------------
Looking for existing binaries...found!
   If for some reason you need to rebuild the binaries, use this command:
      install_sdpt3 -rebuild
---------------------------------------------------------------------------
Adding SDPT3 to the Matlab path:
   Base...already there.
   Solver...already there.
   HSDSolver...already there.
   Binaries...already there.
   Examples...already there.
---------------------------------------------------------------------------
SDPT3 has been succesfully installed.
For more information, type "help sdpt3" or see the user guide.
-------

# Finding an upper bound on CHSH, symmetrized version
During the lecture, we found a form of the SDP that had only one optimization variable. We'll now see how to symmetrize the problem completely.

We first declare the one and only variable, before constructing the moment matrix.

In [5]:
y = sdpvar; % y is actually y_A0B0, but we save space by identifying y with it
X = [ 1  0  0  0  0
      0  1  0  y  y
      0  0  1  y -y
      0  y  y  1  0
      0  y -y  0  1];
I_CHSH = 4*y; % it is <A0B0> + <A0B1> + <A1B0> - <A1B1>, and y_A1B1 = -y_A0B0.
optimize(X >= 0, -I_CHSH, sdpsettings('verbose', 0)); % sign change to maximize, and we don't show solver out
double(I_CHSH)


ans =

    2.8284



To help with symmetrization, we'll use an equivalent form of the constraint X, and check the result.

In [6]:
C = eye(5); % identity
A = [0  0  0  0  0
     0  0  0  1  1
     0  0  0  1 -1
     0  1  1  0  0
     0  1 -1  0  0];
X = C + A*y; % equivalent to the constraint above
optimize(X >= 0, -I_CHSH); % sign change to maximize
double(I_CHSH)


 num. of constraints =  1
 dim. of sdp    var  =  5,   num. of sdp  blk  =  1
*******************************************************************
   SDPT3: Infeasible path-following algorithms
*******************************************************************
 version  predcorr  gam  expon  scale_data
   HKM      1      0.000   1        0    
it pstep dstep pinfeas dinfeas  gap      prim-obj      dual-obj    cputime
-------------------------------------------------------------------
 0|0.000|0.000|8.0e-01|6.2e+00|5.0e+02| 5.000000e+01  0.000000e+00| 0:0:00| chol  1  1 
 1|1.000|1.000|0.0e+00|6.9e-02|4.3e+01| 3.984640e+01  4.400000e-01| 0:0:00| chol  1  1 
 2|0.933|1.000|1.8e-16|6.9e-03|2.9e+00| 3.700415e+00  8.665596e-01| 0:0:00| chol  1  1 
 3|1.000|0.949|1.8e-16|1.0e-03|2.8e-01| 3.024905e+00  2.752468e+00| 0:0:00| chol  1  1 
 4|0.987|0.988|1.8e-16|8.0e-05|3.4e-03| 2.830875e+00  2.827762e+00| 0:0:00| chol  1  1 
 5|0.989|0.989|0.0e+00|7.7e-06|3.8e-05| 2.828454e+00  2.828448e+00| 0:

Now, we want to find a change of basis that brings X into a block-diagonal form. This is easier to check on the basis matrices `C` and `A`, i.e. we want to find `U` such that `U*C*U'` and `U*A*U'` are block-diagonal. Let us use RepLAB for that.
First, remark that the moment matrix is invariant under the following signed permutations:

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, B_0, B_1, A_0, A_1 \right) $

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, -A_0, -A_1, -B_0, -B_1 \right) $

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, A_1, A_0, B_0, -B_1 \right) $

We now show how to use RepLAB to find the change of basis for the symmetry group that includes only the first symmetry.

In [7]:
g1 = [1 4 5 2 3]; % permutation of parties
g2 = [1 -2 -3 -4 -5]; % sign flip everywhere, note the signed permutation convention
g3 = [1 3 2 4 -5]; % additional symmetry

We build the symmetry group from those generators, which are signed permutations on five elements. For the permutation of parties only, the order (=size) of the group is 2. When adding the other symmetries, the order of the group should be 16.

In [8]:
nElements = 5;
G = replab.SignedPermutations(nElements).subgroup({g1 g2 g3});
G.order

ans =
    16


If necessary, we can also expand the group elements (use `G.elements.at(2)` to get the second element for example).

In [9]:
G.elements


ans = 

Enumerator of 16 elements
 1 = [1, 2, 3, 4, 5]    
 2 = [1, -3, -2, -4, 5] 
 3 = [1, -5, -4, -2, 3] 
 4 = [1, 4, 5, 2, 3]    
 5 = [1, -3, 2, 5, -4]  
 6 = [1, -2, 3, -5, -4] 
 7 = [1, 4, -5, 3, 2]   
 8 = [1, 5, -4, -3, 2]  
 9 = [1, -2, -3, -4, -5]
10 = [1, 3, 2, 4, -5]   
11 = [1, 5, 4, 2, -3]   
12 = [1, -4, -5, -2, -3]
13 = [1, 3, -2, -5, 4]  
14 = [1, 2, -3, 5, 4]   
15 = [1, -4, 5, -3, -2] 
16 = [1, -5, 4, 3, -2]  


Now, the representatio we need is composed of signed permutation matrices; as we are considering a group of signed permutations, this is the group *natural representation*. Representations in RepLAB are described using the images of the generators. By calling `G.sampleUniformly`, we get random elements from the group, and by calling `rep.image(g)` we get the image of a group element.

In [10]:
rep = G.naturalRep


rep = 

Orthogonal real representation of dimension 5
dimension: 5                                                                              
    field: 'R'                                                                            
    group: replab.SignedPermutationSubgroup                                               
images{1}: [1, 0, 0, 0, 0; 0, 0, 0, 1, 0; 0, 0, 0, 0, 1; 0, 1, 0, 0, 0; 0, 0, 1, 0, 0]    
images{2}: [1, 0, 0, 0, 0; 0, -1, 0, 0, 0; 0, 0, -1, 0, 0; 0, 0, 0, -1, 0; 0, 0, 0, 0, -1]
images{3}: [1, 0, 0, 0, 0; 0, 0, 1, 0, 0; 0, 1, 0, 0, 0; 0, 0, 0, 1, 0; 0, 0, 0, 0, -1]   


In [11]:
g = G.sampleUniformly
full(rep.image(g)) % we use full, as signed permutation representations return sparse matrices


g =

     1     3     2     4    -5


ans =

     1     0     0     0     0
     0     0     1     0     0
     0     1     0     0     0
     0     0     0     1     0
     0     0     0     0    -1



Now, we decompose the representation into irreducible components. For that, we call `rep.decomposition`. In the answer I(m)xR(d), we express that the component has $m$ copies (multiplicity) of a `R`eal representation of dimension `d`. We then play with the indices of the component, and copy.

Hint: explore `rep.decomposition.component(i).copy(j)` for $(i,j) = (1,1), (1,2), (1,3), (2,1), (2,2)$.

In [12]:
rep.decomposition


ans = 

replab.Irreducible
      parent: Orthogonal real representation of dimension 5
component(1): Isotypic component R(1)                      
component(2): Isotypic component R(2)                      
component(3): Isotypic component R(2)                      


In [13]:
rep.decomposition.component(1).copy(1)


ans = 

Real-type real irreducible subrepresentation
          dimension: 1                                            
              field: 'R'                                          
              group: replab.SignedPermutationSubgroup             
             parent: Orthogonal real representation of dimension 5
realDivisionAlgebra: R                                            
                 U': [1, 0, 0, 0, 0]'                             


We now ask for the change of basis matrix, and verify that `A` and `C` block-diagonalize.

In [14]:
U = full(rep.decomposition.rep.U)
U*C*U'
U*A*U'


U =

    1.0000         0         0         0         0
         0   -0.5000    0.5000         0    0.7071
         0    0.5000    0.5000   -0.7071         0
         0    0.5000   -0.5000         0    0.7071
         0    0.5000    0.5000    0.7071         0


ans =

    1.0000         0         0         0         0
         0    1.0000    0.0000   -0.0000   -0.0000
         0    0.0000    1.0000   -0.0000   -0.0000
         0   -0.0000   -0.0000    1.0000    0.0000
         0   -0.0000   -0.0000    0.0000    1.0000


ans =

         0         0         0         0         0
         0   -1.4142   -0.0000         0    0.0000
         0   -0.0000   -1.4142   -0.0000         0
         0         0   -0.0000    1.4142    0.0000
         0    0.0000         0    0.0000    1.4142



Once we have found the change of basis matrix, we transform our moment matrix `X`, and naturally it should have a block diagonal structure following the one of `C` and `A` -- for that, it may be necessary to kill small off-block coefficients.

In [15]:
X = (U*C*U') + (U*A*U')*y; % equivalent to the constraint above
optimize(X >= 0, -I_CHSH); % sign change to maximize
double(I_CHSH)


 num. of constraints =  1
 dim. of sdp    var  =  5,   num. of sdp  blk  =  1  1th semidefinite block is actually diagonal

*******************************************************************
   SDPT3: Infeasible path-following algorithms
*******************************************************************
 version  predcorr  gam  expon  scale_data
   HKM      1      0.000   1        0    
it pstep dstep pinfeas dinfeas  gap      prim-obj      dual-obj    cputime
-------------------------------------------------------------------
 0|0.000|0.000|8.0e-01|6.2e+00|5.0e+02| 5.000000e+01  0.000000e+00| 0:0:00| chol  1  1 
 1|1.000|1.000|6.5e-08|6.9e-02|4.3e+01| 3.984640e+01  4.400004e-01| 0:0:00| chol  1  1 
 2|0.933|1.000|1.5e-07|6.9e-03|2.9e+00| 3.700414e+00  8.665599e-01| 0:0:00| chol  1  1 
 3|1.000|0.949|3.5e-08|1.0e-03|2.8e-01| 3.024905e+00  2.752468e+00| 0:0:00| chol  1  1 
 4|0.987|0.988|1.1e-08|8.0e-05|3.4e-03| 2.830875e+00  2.827762e+00| 0:0:00| chol  1  1 
 5|0.989|0.989|3.5e-10|7

As an exercice, recover an algebraic basis from `U`, guess the form of the semidefinite program when fully block-diagonal (i.e. for the full group of CHSH symmetries).
**Remark that now the problem can be solved by hand to recover the $2\sqrt{2}$ bound, as the SDP matrix is fully diagonal, and thus corresponds to linear inequalities.**